In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.1/112.6 GB disk)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')  # Kết nối Google Drive

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/train_20241023.zip"

'/content/drive/My Drive/train_20241023.zip'


In [ ]:
!ls '/content/train_data'

daytime  nighttime


In [ ]:
import os
import shutil

# Đường dẫn đến các thư mục daytime và nighttime
daytime_images_dir = '/content/train_data/daytime'  # Đường dẫn đến thư mục daytime
nighttime_images_dir = '/content/train_data/nighttime'  # Đường dẫn đến thư mục nighttime

# Thư mục đích lưu dữ liệu hợp nhất
combined_images_dir = '/content/drive/MyDrive/your_project/combined_dataset/images'
combined_labels_dir = '/content/drive/MyDrive/your_project/combined_dataset/labels'

# Tạo thư mục lưu dữ liệu hợp nhất nếu chưa có
os.makedirs(combined_images_dir, exist_ok=True)
os.makedirs(combined_labels_dir, exist_ok=True)

# Hàm chuyển đổi nhãn cho nighttime
def convert_label(line):
    parts = line.strip().split()
    label = int(parts[0])

    # Chuyển đổi nhãn
    if label == 4:
        label = 0
    elif label == 5:
        label = 1
    elif label == 6:
        label = 2
    elif label == 7:
        label = 3

    # Tạo lại dòng label đã chuyển đổi
    parts[0] = str(label)
    return ' '.join(parts)

# Hàm sao chép và chuyển đổi file
def process_files(images_dir, convert_labels=False):
    for file_name in os.listdir(images_dir):
        if file_name.endswith('.jpg'):
            # Tên file ảnh và label tương ứng
            image_file = file_name
            label_file = file_name.replace('.jpg', '.txt')

            # Sao chép file ảnh vào thư mục combined
            shutil.copy(os.path.join(images_dir, image_file), os.path.join(combined_images_dir, image_file))

            # Đọc và xử lý file label nếu có
            if os.path.exists(os.path.join(images_dir, label_file)):
                with open(os.path.join(images_dir, label_file), 'r') as f:
                    lines = f.readlines()

                # Nếu convert_labels là True, chuyển đổi nhãn
                if convert_labels:
                    lines = [convert_label(line) for line in lines]

                # Lưu label vào thư mục combined
                with open(os.path.join(combined_labels_dir, label_file), 'w') as f:
                    f.writelines(lines)

# Xử lý và sao chép dữ liệu daytime và nighttime
process_files(daytime_images_dir, convert_labels=False)
process_files(nighttime_images_dir, convert_labels=True)

print("Dữ liệu đã được chuyển đổi và gộp thành công!")


Dữ liệu đã được chuyển đổi và gộp thành công!


In [ ]:
!pip install scikit-learn

In [ ]:
import os
import shutil
from sklearn.model_selection import KFold

# Đường dẫn đến dữ liệu
images_dir = '/content/drive/MyDrive/your_project/combined_dataset/images'
labels_dir = '/content/drive/MyDrive/your_project/combined_dataset/labels'
k = 5  # Số lượng folds
output_base_dir = '/content/drive/MyDrive/your_project/cross_validation_folds'

# Tạo KFold cho Cross Validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)
image_files = sorted([f for f in os.listdir(images_dir) if f.endswith('.jpg')])

# Chia dữ liệu thành các folds
for fold, (train_idx, val_idx) in enumerate(kf.split(image_files)):
    fold_dir = os.path.join(output_base_dir, f'fold_{fold}')
    os.makedirs(fold_dir, exist_ok=True)

    train_images_dir = os.path.join(fold_dir, 'train/images')
    train_labels_dir = os.path.join(fold_dir, 'train/labels')
    val_images_dir = os.path.join(fold_dir, 'val/images')
    val_labels_dir = os.path.join(fold_dir, 'val/labels')

    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    os.makedirs(val_labels_dir, exist_ok=True)

    # Copy train data
    for idx in train_idx:
        img_file = image_files[idx]
        label_file = img_file.replace('.jpg', '.txt')

        # Kiểm tra nếu file ảnh và file nhãn đều tồn tại
        if not os.path.exists(os.path.join(images_dir, img_file)):
            print(f"File ảnh {img_file} không tồn tại, bỏ qua.")
            continue
        if not os.path.exists(os.path.join(labels_dir, label_file)):
            print(f"File nhãn {label_file} không tồn tại, bỏ qua.")
            continue

        # Sao chép file vào thư mục train
        shutil.copy(os.path.join(images_dir, img_file), os.path.join(train_images_dir, img_file))
        shutil.copy(os.path.join(labels_dir, label_file), os.path.join(train_labels_dir, label_file))

    # Copy val data
    for idx in val_idx:
        img_file = image_files[idx]
        label_file = img_file.replace('.jpg', '.txt')

        # Kiểm tra nếu file ảnh và file nhãn đều tồn tại
        if not os.path.exists(os.path.join(images_dir, img_file)):
            print(f"File ảnh {img_file} không tồn tại, bỏ qua.")
            continue
        if not os.path.exists(os.path.join(labels_dir, label_file)):
            print(f"File nhãn {label_file} không tồn tại, bỏ qua.")
            continue

        # Sao chép file vào thư mục val
        shutil.copy(os.path.join(images_dir, img_file), os.path.join(val_images_dir, img_file))
        shutil.copy(os.path.join(labels_dir, label_file), os.path.join(val_labels_dir, label_file))

    # Tạo file .yaml cho YOLOv8
    with open(os.path.join(fold_dir, 'data.yaml'), 'w') as f:
        f.write(f'train: {train_images_dir}\n')
        f.write(f'val: {val_images_dir}\n')
        f.write(f'nc: 4\n')  # Số lớp của bạn
        f.write(f'names: ["0", "1", "2", "3"]\n')


In [ ]:
from ultralytics import YOLO
import os
import shutil

# Thiết lập đường dẫn để lưu model tốt nhất và checkpoint
output_base_dir = '/content/drive/MyDrive/your_project/cross_validation_folds'
best_model_path = '/content/drive/MyDrive/your_project/best_model.pt'
best_metric = float('-inf')  # Sử dụng mAP làm tiêu chí đánh giá tốt nhất

# Số lượng folds và epochs
k = 5
epochs = 12

# Khởi động huấn luyện với từng fold
for fold in range(k):
    data_yaml = f'{output_base_dir}/fold_{fold}/data.yaml'

    # Đường dẫn checkpoint để lưu và kiểm tra
    checkpoint_dir = f'{output_base_dir}/checkpoints/fold_{fold}'
    os.makedirs(checkpoint_dir, exist_ok=True)
    last_checkpoint = os.path.join(checkpoint_dir, 'last.pt')

    # Kiểm tra nếu đã có checkpoint để tiếp tục từ đó
    if os.path.exists(last_checkpoint):
        print(f"Tiếp tục huấn luyện từ checkpoint của fold {fold} tại {last_checkpoint}")
        model = YOLO(last_checkpoint)  # Tải model từ checkpoint
        resume_training = True
    else:
        print(f"Bắt đầu huấn luyện mới cho fold {fold}")
        model = YOLO('yolov8n.yaml')  # Tạo model mới
        resume_training = False

    # Huấn luyện model
    results = model.train(
        data=data_yaml,
        epochs=epochs,
        imgsz=640,
        project='/content/drive/MyDrive/your_project/yolo_fold_results',  # Thư mục lưu kết quả
        name=f'fold_{fold}',
        save=True,
        save_period=1,  # Lưu checkpoint mỗi epoch
        resume=resume_training  # Tiếp tục từ checkpoint nếu có
    )

    # Kiểm tra nếu model hiện tại tốt hơn model tốt nhất và lưu lại nếu đúng
    current_metric = results.metrics.mAP50  # mAP ở ngưỡng IoU 0.5
    if current_metric > best_metric:
        best_metric = current_metric
        shutil.copy(results.best, best_model_path)  # Lưu model tốt nhất vào Google Drive

print("Huấn luyện hoàn tất! Model tốt nhất đã được lưu tại:", best_model_path)


Bắt đầu huấn luyện mới cho fold 0
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/your_project/cross_validation_folds/fold_0/data.yaml, epochs=12, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/your_project/yolo_fold_results, name=fold_011, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

train: Scanning /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/labels.cache... 10291 images, 1 backgrounds, 0 corrupt: 100%|██████████| 10292/10292 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/images/cam_02_00316.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/images/cam_02_00565.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/images/cam_02_00579.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/images/cam_02_00951.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/images/cam_04_00014.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/images/cam_04_00018.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/train/images/cam_04_00022.jpg: 1 duplicate labels removed
train:


val: Scanning /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/labels.cache... 2304 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2304/2304 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/images/cam_01_00317.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/images/cam_01_00857.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/images/cam_01_00961.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/images/cam_04_00014.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/images/cam_04_00016.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/images/cam_04_00110.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyDrive/your_project/cross_validation_folds/fold_0/val/images/cam_04_00536.jpg: 1 duplicate labels removed
val: WARNING ⚠️ /content/drive/MyD

...

       6/12      2.77G       1.46     0.8984      1.187         28        640: 100%|██████████| 644/644 [04:51<00:00,  2.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:32<00:00,  2.20it/s]


                   all       2304      20276      0.784      0.714      0.796      0.478

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       7/12      3.15G      1.427     0.8649      1.165         32        640: 100%|██████████| 644/644 [04:55<00:00,  2.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [00:34<00:00,  2.07it/s]


                   all       2304      20276      0.817      0.731      0.827      0.499

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       8/12      2.52G      1.443     0.8786       1.16        133        640:   2%|▏         | 12/644 [00:05<06:17,  1.67it/s]